In [1]:
import numpy as np
import torch
import math
import matplotlib.pyplot as plt
import sklearn
import data_process
import define_model

In [3]:
sbs = ["006","007","009","015","020","030","040","050", "100", "150", "250"]
x_dict = dict.fromkeys(sbs, 0)
y_dict = dict.fromkeys(sbs, 0)

x_ref = np.load('new_separated_data/innerdata_extrasig_train.npy')
print(len(x_ref))

for sb in sbs:
    x_dict[sb] = np.load("data_SB_" + sb + '/innerdata_extrasig_train.npy')
    print(len(x_dict[sb]))
    # y_dict[sb] = np.load("X_files/y_train"+sb+".npy")



27263
27286
27226
27108
26756
26458
25863
25273
24674
21699
18746
12869
